In [1]:
key = ''

In [2]:
import pandas as pd
import re
import ast
import json
import string
import requests
from tqdm import tqdm
import time

C:\Users\derya\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [8]:
questions = pd.read_csv('C:/Users/derya/Documents/DSBA/MENTALISM/tweets_gen/Eng_questions.csv', sep=';')
questions_list = questions['Question'].tolist()

# Function that adapts to model temperature 

In [7]:
import openai
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential
)

# Define the GPT classification function
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def gpt_classify(question_id, messages, temperature):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        temperature=temperature,
        messages=messages
    )
    answer = response.choices[0]['message']['content']
    return answer


In [10]:
def tweets_gen_temp(questions_list, temperature):
    try:
        
        transformed_temp = str(temperature).replace('.', '_')
        name = f'temp_{transformed_temp}'
        column_names = ['Question', 'Measure', 'Gen\Ref', 'Positive Tweets', 'Negative Tweets']
        dfs = {}
        dfs[name]= pd.DataFrame(columns=column_names)
        
        items = list(range(len(questions_list)))
        progress_bar = tqdm(items, desc="Progress", leave=True)
                

        for i in progress_bar:

            question = questions_list[i]

            prompt_start_pop = F"""

            Step 1: I will show you a question: {question}. Write written what can be measured with this 
            question in social questionere maximum in two words.

            Step 2: Provide 5 positive possible tweets people could write on this topic. 

            Step 3: For each tweet created in previous create the opposite in meaning. 

            Step 4: Provide 5 possible negative tweets people could write on this topic.  

            Step 5: For each tweet created in previous create the opposite in meaning. 

            reply in a form:

            'Step 1: step 1 result \n\nStep 2: \n. sentence. \n. sentence. \n. sentence. \n. sentence.\n. sentance.\n\n
            Step 3:\n. sentence.\n. sentence.\n. sentence.\n.  sentence.\n. sentence.\n\nStep 4:\n. sentence. \n.
            2nd sentence.\n. sentence.\n. sentence. \n. sentence.\n\nStep 5:\n. sentence.\n. sentence.\n. 
            sentence.\n. sentence.\n. sentence.'

            do not enumerate sentences
            """

            messages = [
            {"role": "system", "content": "You are a twitter user."},
            {"role": "user", "content": prompt_start_pop}
            ]

            openai.api_key = key
            question_id = 123

            get_answer = gpt_classify(question_id, messages, temperature)


            get_answer = get_answer.strip("'")
            steps = re.findall(r"Step \d+:((?:.|\n(?!\n))+)", get_answer)
            steps = [step.strip() for step in steps]

            result_lists = {}
            for j in range(1, len(steps)):
                list_name = f"list_{j}"
                sentences = steps[j].split('\n')
                result_lists[list_name] = sentences


            ## DF preparation

            data_pos = [
                [steps[0], 'Generated'] +  [result_lists['list_1']],
                [steps[0], 'Reflected'] + [result_lists['list_4']]
            ]
            columns_pos = ['Measure', 'Gen\Ref', 'Positive Tweets']
            df_pos = pd.DataFrame(data_pos, columns=columns_pos)
            df_pos = df_pos.explode('Positive Tweets')

            df_pos.to_csv(f'df_pos_{i}.csv', index = False)


            data_neg = [
                [steps[0], 'Generated'] +  [result_lists['list_3']],
                [steps[0], 'Reflected'] + [result_lists['list_2']]
            ]
            columns_neg = ['Measure', 'Gen\Ref', 'Negative Tweets']
            df_neg = pd.DataFrame(data_neg, columns=columns_neg)
            df_neg = df_neg.explode('Negative Tweets')


            df_pos['Negative Tweets'] = df_neg['Negative Tweets']
            df_pos['Question'] = question

            dfs[name] = pd.concat([dfs[name], df_pos], ignore_index=True)
            
            progress_bar.set_postfix({"Item": i}, refresh=True)

        progress_bar.close()
        dfs[name].to_csv(f'tweets_gen_{name}.csv', index = False)
        return dfs[name]
    
    except Exception:
        return "Apperently you set temperature too high and chatGPT created something gibberish"


In [30]:
tweets_gen_temp(questions_list, 0)

Progress: 100%|██████████| 14/14 [02:30<00:00, 10.78s/it, Item=13]


,Question,Measure,Gen\Ref,Positive Tweets,Negative Tweets
0,For how long in the past 4 weeks have you felt...,Measuring calmness/serenity in the past 4 weeks.,Generated,"1. ""Feeling so peaceful and content lately, it...","1. ""Can't remember the last time I felt calm o..."
1,For how long in the past 4 weeks have you felt...,Measuring calmness/serenity in the past 4 weeks.,Generated,"2. ""Finding solace in the little moments of tr...","2. ""Feeling constantly on edge, no peace in si..."
2,For how long in the past 4 weeks have you felt...,Measuring calmness/serenity in the past 4 weeks.,Generated,"3. ""Grateful for the moments of calmness that ...","3. ""Stress and anxiety have taken over, no roo..."
3,For how long in the past 4 weeks have you felt...,Measuring calmness/serenity in the past 4 weeks.,Generated,"4. ""Embracing the serenity and letting go of s...","4. ""Every day feels like a battle, no time for..."
4,For how long in the past 4 weeks have you felt...,Measuring calmness/serenity in the past 4 weeks.,Generated,"5. ""Feeling a sense of inner peace that has be...","5. ""Feeling trapped in a never-ending cycle of..."
...,...,...,...,...,...
135,In the next 5 years do you think your personal...,"Personal situation in 5 years - Improve, Same,...",Reflected,1. My personal situation is bound to improve. ...,1. Feeling pessimistic about the future. #Lack...
136,In the next 5 years do you think your personal...,"Personal situation in 5 years - Improve, Same,...",Reflected,2. Confident that positive changes are on the ...,2. Worried that my personal situation will sta...
137,In the next 5 years do you think your personal...,"Personal situation in 5 years - Improve, Same,...",Reflected,"3. No room for setbacks, only progress in the ...",3. Unsure if I can make any progress in improv...
138,In the next 5 years do you think your personal...,"Personal situation in 5 years - Improve, Same,...",Reflected,4. Breaking free from negativity and embracing...,4. Doubting my ability to create a better futu...


In [31]:
tweets_gen_temp_0 = pd.read_csv('C:/Users/derya/Documents/DSBA/MENTALISM/tweets_gen/tweets_gen_temp_0.csv')

# Code without adaptation to model temperature 

In [292]:
## For all questions
column_names = ['Question', 'Measure', 'Gen\Ref', 'Positive Tweets', 'Negative Tweets']
output = pd.DataFrame(columns=column_names)

for i in range (0,len(questions_list)):

    question = questions_list[i]

    prompt_start_pop = F"""

    Step 1: I will show you a question: {question}. Write written what can be measured with this 
    question in social questionere maximum in two words.

    Step 2: Provide 5 positive possible sentences people could write on this topic. 

    Step 3: Analyze previously created 5 positive sentences written on this topic and provide for each sentimental
    opposite of them.

    Step 4: Provide 5 possible negative sentences people could write on this topic.  

    Step 5: Analyze previously created 5 negative sentences written on this topic and provide for each sentimental 
    opposite of them.

    reply in a form:

    'Step 1: step 1 result \n\nStep 2: \n. 1st sentence. \n. 2nd sentence. \n. 3d sentence. \n. 4th sentence.\n. 5th sentance.\n\n
    Step 3:\n. 1st sentence.\n. 2nd sentence.\n. 3d sentence.\n.  4th sentence.\n. 5th sentence.\n\nStep 4:\n. 1st sentence. \n.
    2nd sentence.\n. 3d sentence.\n. 4th sentence. \n. 5th sentence.\n\nStep 5:\n. 1st sentence.\n. 2nd sentence.\n. 3d 
    sentence.\n. 4th sentence.\n. 5th sentence.'

    """

    messages = [
    {"role": "system", "content": "You are a twitter user."},
    {"role": "user", "content": prompt_start_pop}
    ]

    openai.api_key = key
    question_id = 123

    get_answer = gpt_classify(question_id, messages)
    get_answer = get_answer.strip("'")
    steps = re.findall(r"Step \d+:((?:.|\n(?!\n))+)", get_answer)
    steps = [step.strip() for step in steps]

    result_lists = {}
    for i in range(1, len(steps)):
        list_name = f"list_{i}"
        sentences = re.split(r'\d+\.\s*', steps[i])[1:]
        sentences = [sentence.strip() for sentence in sentences]
        result_lists[list_name] = sentences
    
    ## DF preparation

    data_pos = [
        [steps[0], 'Generated'] +  [result_lists['list_1']],
        [steps[0], 'Reflected'] + [result_lists['list_4']]
    ]
    columns_pos = ['Measure', 'Gen\Ref', 'Positive Tweets']
    df_pos = pd.DataFrame(data_pos, columns=columns_pos)
    df_pos = df_pos.explode('Positive Tweets')


    data_neg = [
        [steps[0], 'Generated'] +  [result_lists['list_3']],
        [steps[0], 'Reflected'] + [result_lists['list_2']]
    ]
    columns_neg = ['Measure', 'Gen\Ref', 'Negative Tweets']
    df_neg = pd.DataFrame(data_neg, columns=columns_neg)
    df_neg = df_neg.explode('Negative Tweets')

    df_pos['Negative Tweets'] = df_neg['Negative Tweets']
    que = pd.DataFrame(questions_list, columns=['Question'])
    df_pos['Question'] = que['Question'][i]
    
    output = pd.concat([output, df_pos], ignore_index=True)

output   

,Question,Measure,Gen\Ref,Positive Tweets,Negative Tweets
0,For how long in the past 4 weeks have you felt...,Calmness/Serenity in 4 weeks,Generated,I've been feeling calm and serene for most of ...,I haven't felt calm or serene in weeks.
1,For how long in the past 4 weeks have you felt...,Calmness/Serenity in 4 weeks,Generated,My daily meditation practice has helped me fee...,My stress levels have been through the roof la...
2,For how long in the past 4 weeks have you felt...,Calmness/Serenity in 4 weeks,Generated,Spending time in nature always brings me a sen...,I feel like I'm constantly on edge and can't s...
3,For how long in the past 4 weeks have you felt...,Calmness/Serenity in 4 weeks,Generated,I've been practicing deep breathing exercises ...,The chaos and uncertainty of the world right n...
4,For how long in the past 4 weeks have you felt...,Calmness/Serenity in 4 weeks,Generated,Taking breaks throughout the day to stretch an...,I've been dealing with a lot of personal drama...
...,...,...,...,...,...
135,For how long in the past 4 weeks have you felt...,Personal situation in 5 years,Reflected,I am confident that my personal situation will...,I am worried that my personal situation will s...
136,For how long in the past 4 weeks have you felt...,Personal situation in 5 years,Reflected,I am optimistic about my future and expect my ...,I am unsure if my personal situation will impr...
137,For how long in the past 4 weeks have you felt...,Personal situation in 5 years,Reflected,I am hopeful that my personal situation will i...,I am not confident that my personal situation ...
138,For how long in the past 4 weeks have you felt...,Personal situation in 5 years,Reflected,I am excited about the possibilities for my pe...,I am skeptical that my personal situation will...


In [293]:
output.to_csv('generated_tweets_ref.csv', index=False)